# 🚀 Caso aplicado: Agrupar start-ups por éxito e innovación
**Técnica:** K-Means Clustering**

**Objetivo:** Segmentar start-ups según su desempeño, innovación y sector tecnológico para identificar patrones de éxito.

## 0️⃣ Preparación del entorno
**Objetivo:** Cargar librerías y fijar semilla para reproducibilidad.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

np.random.seed(42)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

## 1️⃣ Simulación de datos
**Objetivo:** Generar base simulada de start-ups con métricas de innovación y éxito.

In [ ]:
n = 300
sectores = np.random.choice(['IA', 'Fintech', 'Salud', 'EdTech', 'E-commerce', 'ClimaTech'], size=n, p=[0.18, 0.20, 0.16, 0.16, 0.20, 0.10])
etapa = np.random.choice(['Pre-Seed', 'Seed', 'Series A', 'Series B'], size=n, p=[0.25, 0.35, 0.25, 0.15])

sector_factor = pd.Series(sectores).map({'IA': 1.15, 'Fintech': 1.10, 'Salud': 1.05, 'EdTech': 0.95, 'E-commerce': 1.00, 'ClimaTech': 1.08}).values
innovacion = np.clip(np.random.beta(a=2, b=2, size=n) * sector_factor, 0, 1)
uso_ia = ((sectores=='IA') | (np.random.rand(n) < 0.25)).astype(int)
modelo_saas = (np.random.rand(n) < 0.55).astype(int)
acelerada = (np.random.rand(n) < 0.35).astype(int)

ingresos_mensuales = np.random.lognormal(mean=10, sigma=0.6, size=n) * (0.8 + 0.4*modelo_saas)
crec_ingresos = np.random.normal(loc=0.08, scale=0.06, size=n) + 0.04*innovacion
usuarios_mensuales = np.random.lognormal(mean=11, sigma=0.8, size=n) * (1 + 0.3*uso_ia)
crec_usuarios = np.random.normal(loc=0.10, scale=0.07, size=n) + 0.03*innovacion
churn = np.clip(np.random.normal(loc=0.06, scale=0.03, size=n) - 0.01*modelo_saas - 0.01*innovacion, 0.0, 0.25)
nps = np.clip(np.random.normal(loc=35, scale=15, size=n) + 20*innovacion - 10*churn, -100, 100)

funding_total = np.random.lognormal(mean=15, sigma=0.9, size=n) * (1 + 0.2*acelerada)
burn_mensual = np.random.lognormal(mean=11, sigma=0.7, size=n)
runway_meses = np.clip((funding_total / (burn_mensual + 1e-6)), 2, 36)

tam_equipo = np.clip((np.random.normal(20, 12, n) + 15*(etapa=='Series A') + 25*(etapa=='Series B')), 3, None)
experiencia_fundadores = np.clip(np.random.normal(7, 3.5, n) + 1.5*acelerada + 1.0*(sectores=='Fintech'), 0, 25)
valuacion_proxy = (ingresos_mensuales*12)*(1 + 8*crec_ingresos) * (1 + 0.6*innovacion)

df = pd.DataFrame({
    'sector': sectores, 'etapa': etapa, 'innovacion': innovacion,
    'uso_ia': uso_ia, 'modelo_saas': modelo_saas, 'acelerada': acelerada,
    'ingresos_mensuales': ingresos_mensuales, 'crec_ingresos': crec_ingresos,
    'usuarios_mensuales': usuarios_mensuales, 'crec_usuarios': crec_usuarios,
    'churn': churn, 'nps': nps, 'funding_total': funding_total,
    'burn_mensual': burn_mensual, 'runway_meses': runway_meses,
    'tam_equipo': tam_equipo, 'experiencia_fundadores': experiencia_fundadores,
    'valuacion_proxy': valuacion_proxy
})
df.head()

## 2️⃣ EDA - Exploración inicial de datos

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.select_dtypes('number').corr(), cmap='Blues')
plt.title('Correlaciones numéricas')
plt.show()

## 3️⃣ Preprocesamiento (One-hot + Escalamiento)

In [ ]:
cat_cols = ['sector', 'etapa']
num_cols = [c for c in df.columns if c not in cat_cols]

prepro = ColumnTransformer([
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols),
    ('num', StandardScaler(), num_cols)
], remainder='drop')

X = prepro.fit_transform(df)
X.shape

## 4️⃣ Selección de K (Codo y Silueta)

In [ ]:
inertias = []
sil_vals = {}
K_range = range(2, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)
    labels = km.predict(X)
    sil_vals[k] = silhouette_score(X, labels)

plt.figure(figsize=(6,4))
plt.plot(list(K_range), inertias, marker='o')
plt.title('Método del codo')
plt.xlabel('k'); plt.ylabel('Inercia')
plt.show()

sil_vals

## 5️⃣ Entrenamiento del modelo K-Means

In [ ]:
k_elegido = 5  # Ajustar según resultados
kmeans = KMeans(n_clusters=k_elegido, random_state=42, n_init=10)
labels = kmeans.fit_predict(X)

df_clusters = df.copy()
df_clusters['cluster'] = labels

sil = silhouette_score(X, labels)
print(f'Coeficiente de silueta (k={k_elegido}): {sil:.3f}')
df_clusters.head()

## 6️⃣ Perfilamiento de clústeres

In [ ]:
resumen = df_clusters.groupby('cluster').agg(['mean','median']).round(2)
sector_mix = pd.crosstab(df_clusters['cluster'], df_clusters['sector'], normalize='index').round(2)
etapa_mix = pd.crosstab(df_clusters['cluster'], df_clusters['etapa'], normalize='index').round(2)
resumen.head(10), sector_mix, etapa_mix

## 7️⃣ Visualización de resultados

In [ ]:
vars_clave = ['innovacion','crec_ingresos','nps','churn','runway_meses','valuacion_proxy']
fig, axes = plt.subplots(2, 3, figsize=(14,8))
axes = axes.ravel()
for i, col in enumerate(vars_clave):
    sns.boxplot(data=df_clusters, x='cluster', y=col, ax=axes[i])
    axes[i].set_title(col)
plt.tight_layout(); plt.show()

sector_mix.plot(kind='bar', stacked=True, figsize=(8,5))
plt.title('Composición sectorial por clúster')
plt.ylabel('Proporción'); plt.legend(title='Sector', bbox_to_anchor=(1.05,1))
plt.tight_layout(); plt.show()

## 8️⃣ Recomendaciones estratégicas
Redacta 2–3 estrategias para cada clúster según sus características.

## 9️⃣ Exportar resultados

In [ ]:
df_clusters.to_csv('startups_clusters.csv', index=False)
resumen.to_csv('resumen_kpis_por_cluster.csv')
sector_mix.to_csv('mix_sector_por_cluster.csv')
etapa_mix.to_csv('mix_etapa_por_cluster.csv')